In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 10, 10
import numpy as np
import scipy as sp
from scipy.misc import lena
import cv2
import glob
import pba

### import images , resize them =>  show them 

In [2]:
img_paths = sorted(glob.glob('../data/*.jpg'))
imgs = [cv2.resize(cv2.imread(img_path, cv2.IMREAD_GRAYSCALE), (600, 400)) for img_path in img_paths]
mask = np.ones(shape=(400, 600), dtype=np.uint8)
mask[300:, 250:380] = 0

### find connection between picturs : img1 and img2

In [3]:
def draw_matches(img1, kp1, img2, kp2, matches, color=None): 
    """Draws lines between matching keypoints of two images.  
    Keypoints not in a matching pair are not drawn.
    Places the images side by side in a new image and draws circles 
    around each keypoint, with line segments connecting matching pairs.
    You can tweak the r, thickness, and figsize values as needed.
    Args:
        img1: An openCV image ndarray in a grayscale or color format.
        kp1: A list of cv2.KeyPoint objects for img1.
        img2: An openCV image ndarray of the same format and with the same 
        element type as img1.
        kp2: A list of cv2.KeyPoint objects for img2.
        matches: A list of DMatch objects whose trainIdx attribute refers to 
        img1 keypoints and whose queryIdx attribute refers to img2 keypoints.
        color: The color of the circles and connecting lines drawn on the images.  
        A 3-tuple for color images, a scalar for grayscale images.  If None, these
        values are randomly generated.  
    """
    # We're drawing them side by side.  Get dimensions accordingly.
    # Handle both color and grayscale images.
    if len(img1.shape) == 3:
        new_shape = (max(img1.shape[0], img2.shape[0]), img1.shape[1]+img2.shape[1], img1.shape[2])
    elif len(img1.shape) == 2:
        new_shape = (max(img1.shape[0], img2.shape[0]), img1.shape[1]+img2.shape[1])
        
    new_img = np.zeros(new_shape, type(img1.flat[0]))  
    
    # Place images onto the new image.
    new_img[0:img1.shape[0],0:img1.shape[1]] = img1
    new_img[0:img2.shape[0],img1.shape[1]:img1.shape[1]+img2.shape[1]] = img2
    
    # Draw lines between matches.  Make sure to offset kp coords in second image appropriately.
    r = 15
    thickness = 2
    if color:
        c = color
        
    for m in matches:
        # Generate random color for RGB/BGR and grayscale images as needed.
        if not color: 
            c = np.random.randint(0,256,3) if len(img1.shape) == 3 else np.random.randint(0,256)
            
        # So the keypoint locs are stored as a tuple of floats.  cv2.line(), like most other things,
        # wants locs as a tuple of ints.
        #print m, dir(m)
        end1 = tuple(np.round(kp1[m.queryIdx].pt).astype(int))
        end2 = tuple(np.round(kp2[m.trainIdx].pt).astype(int) + np.array([img1.shape[1], 0]))
        cv2.line(new_img, end1, end2, c, thickness)
        cv2.circle(new_img, end1, r, c, thickness)
        cv2.circle(new_img, end2, r, c, thickness)
        
    return new_img

In [4]:
def match2(img1, img2, kp_mask):
    # Initiate ORB detector
    orb = cv2.ORB()

    # find the keypoints with ORB
    kp = orb.detect(img1, kp_mask)
    kp1, des1 = orb.compute(img1, kp)
    kp = orb.detect(img2, kp_mask)
    kp2, des2 = orb.compute(img2, kp)

    # FLANN parameters
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)   # or pass empty dictionary

    flann = cv2.FlannBasedMatcher(index_params, search_params)

    matches = flann.knnMatch(des1.astype(np.float32), des2.astype(np.float32), k=2)

    # store all the good matches as per Lowe's ratio test.
    good = []
    for m, n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)

    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    temp = mask.ravel().tolist()

    matches_filtered = [g for g, t in zip(good, temp) if t]

    imgNew = draw_matches(img1, kp1, img2, kp2, matches_filtered)
    plt.figure(figsize=(20,10))
    plt.imshow(imgNew, cmap='gray')
    plt.show()

In [5]:
def match(imgs, kp_mask):
    #
    # Initiate ORB detector
    #
    orb = cv2.ORB()

    kps = []
    descs = []
    for img in imgs:
        #
        # find the keypoints with ORB
        #
        kp = orb.detect(img, kp_mask)
        kp, desc = orb.compute(img, kp)
        kps.append(kp)
        descs.append(desc)
    
    kp_mask = np.zeros((len(imgs), len(kps[0])), dtype=np.uint8)
    kp_coords = np.zeros((len(imgs), len(kps[0]), 2), dtype=np.float)
    
    #
    # FLANN parameters
    #
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    for kp_index, (desc, kp) in enumerate(zip(descs[1:], kps[1:])):
        matches = flann.knnMatch(descs[0].astype(np.float32), desc.astype(np.float32), k=2)
        
        idx_filtered = []
        for i, (m, n) in enumerate(matches):
            if m.distance < 0.7*n.distance:
                idx_filtered.append(i)

        if len(idx_filtered) < 4:
            continue
        
        src_pts = np.float32([kps[0][matches[i][0].trainIdx].pt for i in idx_filtered]).reshape(-1,1,2)
        dst_pts = np.float32([kp[matches[i][0].queryIdx].pt for i in idx_filtered]).reshape(-1,1,2)

        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        
        idx_filtered = [i for i, m in zip(idx_filtered, mask.ravel()) if m]

        kp_mask[0, idx_filtered] = 1
        kp_coords[0, idx_filtered, :] = np.squeeze(src_pts[mask.astype(np.bool), :])

        kp_mask[kp_index+1, idx_filtered] = 1
        kp_coords[kp_index+1, idx_filtered, :] = np.squeeze(dst_pts[mask.astype(np.bool), :])
        
    return kp_mask, kp_coords

In [6]:
kp_mask, kps = match(imgs, None)

In [7]:
vmask = kp_mask[:, np.nonzero(kp_mask.sum(axis=0))[0]]
vmask = np.ascontiguousarray(vmask.T).astype(np.uint8)

In [8]:
measures = kps[:, np.nonzero(kp_mask.sum(axis=0))[0], :]

In [10]:
J, I = np.nonzero(vmask)
measures = measures[I, J]

In [12]:
measures = np.ascontiguousarray(measures)

In [13]:
A, B = pba.run_pba(None, None, measures, vmask)

In [14]:
A

array([[ 999.99993896,    1.        ,    0.        ,    0.        ,
           0.        ,    1.        ,    0.        ,    0.        ,
           0.        ,    1.        ,           nan,           nan,
                  nan],
       [ 999.99993896,    1.        ,    0.        ,    0.        ,
           0.        ,    1.        ,    0.        ,    0.        ,
           0.        ,    1.        ,           nan,           nan,
                  nan],
       [ 999.99993896,    1.        ,    0.        ,    0.        ,
           0.        ,    1.        ,    0.        ,    0.        ,
           0.        ,    1.        ,           nan,           nan,
                  nan],
       [ 999.99993896,    1.        ,    0.        ,    0.        ,
           0.        ,    1.        ,    0.        ,    0.        ,
           0.        ,    1.        ,           nan,           nan,
                  nan],
       [ 999.99993896,    1.        ,    0.        ,    0.        ,
           0.       

In [15]:
import quaternions as qua

In [16]:
cameras = []
with open('cams.txt', 'r') as f:
    for line in f:
        a = np.fromstring(line.strip(), sep=' ')
        f = np.array([a[0]])
        q = a[10:14]
        T = a[14:]
        R = np.array(qua.Quaternion(*q.tolist()).asRotationMatrix())
        cameras.append(np.concatenate((f, R.ravel(), T.ravel())))

In [17]:
cameras = np.concatenate(cameras)

In [18]:
points = []
measurements = []
vi = []
vj = []
j = 0
with open('pts.txt', 'r') as f:
    for line in f:
        a = np.fromstring(line.strip(), sep=' ')
        points.append(a[:3])
        app = int(a[3])
        for i in range(app):
            vi.append(int(a[4+i*3]))
            vj.append(j)
            measurements.append(a[5+i*3:7+i*3])
        j += 1

vmask = np.zeros((len(points), 3), dtype=np.uint8)
vmask[vj, vi] = 1

points = np.concatenate(points)
measurements = np.concatenate(measurements)

In [20]:
A, B = pba.run_pba(cameras, points, measurements, vmask)

In [21]:
A

array([[  1.76290234e+03,  -3.25603217e-01,   6.38462827e-02,
         -3.52472663e-01,   5.46009958e-01,   8.27237964e-01,
         -1.06706157e-01,   7.71915615e-01,  -5.58210552e-01,
         -6.44628406e-01,  -4.38879341e-01,  -2.93108344e-01,
         -1.27592385e+00],
       [  8.12118591e+02,  -4.38807875e-01,   1.13957167e-01,
          3.04009706e-01,  -6.10692024e-01,   6.89808071e-01,
         -2.74147749e-01,  -6.59154594e-01,  -7.14954734e-01,
         -6.03262305e-01,  -4.23521362e-02,  -4.43524212e-01,
         -8.23009431e-01],
       [  4.24639302e-06,   2.91910261e-01,   9.04538691e-01,
          3.10802847e-01,  -9.39500809e-01,   3.32074910e-01,
         -8.40545967e-02,  -1.79240301e-01,  -2.67462879e-01,
          9.46750402e-01,   3.72451757e+12,   4.66002379e+12,
          2.85575950e+06]])

In [22]:
B

array([[  1.58761233e-01,  -6.03281036e-02,  -9.30928767e-01],
       [  1.27083763e-01,  -4.91583981e-02,  -9.39415812e-01],
       [  2.35567894e-02,   2.20771981e-04,  -9.93594825e-01],
       ..., 
       [  8.10253620e-02,   2.84893904e-02,  -5.85538685e-01],
       [  8.13209787e-02,   3.07266116e-02,  -5.87912083e-01],
       [  8.14358145e-02,   3.30847949e-02,  -5.90292096e-01]])